### Notebook to test model for final performance metrics

In [4]:
import sys
import os
import torch
from omegaconf import OmegaConf

sys.path.append(os.path.dirname(os.path.abspath('.')))
from model.dataset import create_dataloaders
from model.model import GMMModel

cfg = OmegaConf.load('/home/lc865/workspace/DL-GNNs/Temporal-Graph-MLP-Mixer/src/TGMM-1/train/metrla.yaml')
train_loader, val_loader, test_loader, topo_data, metadata = create_dataloaders(cfg, raw_data_dir='/mnt/cephfs/store/gr-mc2473/lc865/workspace/GNN/data')

# Initialize model with Lightning
# model = GMMModel(cfg, topo_data, metadata)

# Load model checkpoint using Lightning
checkpoint_path = '/mnt/cephfs/store/gr-mc2473/lc865/workspace/GNN/checkpoints/last-v41.ckpt'  # Todo: Replace with WandB loading
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GMMModel.load_from_checkpoint(checkpoint_path, cfg=cfg, topo_data=topo_data, metadata=metadata, map_location=device)

# Set model to evaluation mode
model.eval()
pass


Dataloaders and Topology Data Overview:
Batch size: 16
----------------------------------------------------------
Name                 | Samples      | Batches     
----------------------------------------------------------
Train Loader         | 23974        | 1498        
Val Loader           | 3424         | 214         
Test Loader          | 6849         | 428         

Topology Data:
Number of nodes: 207
Number of edges: 1722
----------------------------------------------------------


In [5]:
# Run test with Lightning trainer
from lightning import Trainer

trainer = Trainer(
    accelerator='auto',
    devices=1,
    logger=None,
    enable_checkpointing=False
)
torch.set_float32_matmul_precision('medium')

# Run test and get metrics
test_results = trainer.test(model, test_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing DataLoader 0: 100%|██████████| 428/428 [00:15<00:00, 26.79it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/loss           0.2852415144443512
       test/mae-12           5.665010929107666
       test/mae-3            4.081916809082031
       test/mae-6            4.822720527648926
      test/mae-all           4.787268161773682
      test/mape-12          13.482573509216309
       test/mape-3           9.696481704711914
       test/mape-6          11.310423851013184
      test/mape-all           11.286865234375
      test/rmse-12           9.574116706848145
       test/rmse-3           6.958033084869385
       test/rmse-6           8.185614585876465
      test/rmse-all          8.217775344848633
────────────────────────────────